# 既存資料からチャットボットを作成する(Retrieval Augumented Generation:RAG)

- 参考
  - https://colab.research.google.com/github/nyanta012/demo/blob/main/sentence_retrieval.ipynb
  - https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html
  - https://note.com/mahlab/n/nb6677d0fc7c2
- Chromadbがすごい
  - https://www.trychroma.com/
  - https://github.com/chroma-core/chroma
- ChatGPTのSurvey論文
  - https://arxiv.org/pdf/2304.01852.pdf
  


In [1]:
%%capture
# 必要なパッケージをインストール
!pip install -U langchain-community
!pip install openai
# 残りのコードをここに追加
!pip install --upgrade chromadb langchain
!pip install pypdf
!pip install tiktoken
!pip install gradio
!pip install pypdf2
!pip install -U langchain-openai

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader
import openai
import os
# langchain-openai パッケージからインポート
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [3]:
API_KEY="XXXXXXXXXXXXXXXXX"

In [4]:
os.environ["OPENAI_API_KEY"] = API_KEY
openai.api_key = API_KEY

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
import PyPDF2

class Document:
    def __init__(self, text, metadata):
        self.page_content = text
        self.metadata = metadata

dir_path = "/content/drive/MyDrive/ユニークAIシステム構築道場/RAG/loadtext/"

documents = []

for filename in os.listdir(dir_path):
    file_path = os.path.join(dir_path, filename)
    metadata = {'filename': filename}

    # PDFファイルを読み込む
    if filename.endswith('.pdf'):
        with open(file_path, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            text = ''
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
            documents.append(Document(text, metadata))

    # テキストファイルを読み込む
    elif filename.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            documents.append(Document(text, metadata))


In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-4"), chain_type="stuff", retriever=vectordb.as_retriever())

In [8]:
# 質問を作成するために invoke メソッドを使用
questions = qa.invoke("この文章の中で重要ど思う線形代数に関する内容についてを考える質問を10個考えてください")

# 結果の表示
print(questions['result'])

1. 線形代数における「線形写像」は何を意味しますか？
2. ベクトルと行列ではどのような計算や操作が可能ですか？
3. 具体的に、線形変換や行列の積（写像の合成）をする際の計算手順は何ですか？
4. コサイン類似度の計算に線形代数のどの部分が用いられますか？
5. 線形代数における「核(Ker)」とは何ですか？
6. Pythonで線形代数の計算を行う場合、どのようなライブラリやメソッドが利用可能ですか？
7. 線形代数がデータサイエンス等の実用的な応用分野でどのように用いられますか？
8. 「連立一次方程式」はどのように解くことができますか？
9. 「連立一次方程式」をベクトルと行列を用いて表現することは何を可能にしますか？
10. ベクトルや行列、線形変換等の数学的な概念を体系的に学ぶことで得られる利点は何ですか？


In [9]:
# 質問を作成するために invoke メソッドを使用
questions = qa.invoke("疑似逆行列とは何ですか？検索できなければわからないと答えてください。")

# 結果の表示
print(questions['result'])

私の知識からは、疑似逆行列についての情報を提供することはできません。お手数をおかけしますが、それについては別の情報源をご覧になるか、専門家に問い合わせてください。


In [10]:
# インポート
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import VectorDBQA, RetrievalQA
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
import gradio as gr
import openai
import os

# ドキュメントをロード
dir_path = '/content/drive/MyDrive/ユニークAIシステム構築道場/RAG/loadtext/'

documents = []

for filename in os.listdir(dir_path):
    file_path = os.path.join(dir_path, filename)
    metadata = {'filename': filename}

    # PDFファイルを読み込む
    if filename.endswith('.pdf'):
        with open(file_path, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            text = ''
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
            documents.append(Document(text, metadata))

    # テキストファイルを読み込む
    elif filename.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            documents.append(Document(text, metadata))


# ドキュメントの分割とベクトルDBの作成
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(texts, embeddings)

# QAチェーンの作成
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-4"), chain_type="stuff", retriever=vectordb.as_retriever())

# チャット機能の実装
def chat_with_ai(input_text):
    response = qa.invoke(input_text+'検索できた範囲で答えてください。検索できないものは「わかりません」と答えてください。')
    return response['result']

# Gradioインターフェースの設定
iface = gr.Interface(
    fn=chat_with_ai,
    inputs="text",
    outputs="text",
    title="AI Chat with Gradio",
    description="OpenAIを使ったチャットインターフェース"
)

# インターフェースの起動
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d070f60ce52cdb6e86.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# インポート
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import VectorDBQA, RetrievalQA
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
import gradio as gr
import openai
import os

# ドキュメントをロードするクラス
class Document:
    def __init__(self, text, metadata):
        self.page_content = text
        self.metadata = metadata

# ドキュメントをロード（例としてテキストを使用）
dir_path = '/content/drive/MyDrive/Colab Notebooks/CollaborativeResearch/BandaiLogipal/BasicChat/loadtext2'

documents = []

for filename in os.listdir(dir_path):
    file_path = os.path.join(dir_path, filename)
    metadata = {'filename': filename}

    # PDFファイルを読み込む
    if filename.endswith('.pdf'):
        with open(file_path, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            text = ''
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
            documents.append(Document(text, metadata))

    # テキストファイルを読み込む
    elif filename.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            documents.append(Document(text, metadata))

# ドキュメントの分割とベクトルDBの作成
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(texts, embeddings)

# QAチェーンの作成
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-4"), chain_type="stuff", retriever=vectordb.as_retriever())

# チャット機能の実装
def chat_with_ai(input_text, history):
    response = qa({"query": input_text+'検索できた範囲で答えてください。検索できないものは「わかりません」と答えてください。'})
    return response['result']

# GradioのChatInterfaceを使用してインターフェースを作成
with gr.Blocks() as demo:
    chatbot = gr.ChatInterface(fn=chat_with_ai, chatbot=gr.Chatbot())
    gr.Markdown("### AI Chat with Gradio")

# インターフェースの起動
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://135cc0aa561ecf76f8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
